# VacationPy
----

In this notebook the recently created csv file with weather information of  56?? cities around the globe is going to be used. The main idea is to finf the best places to have an idillyc vacation time. But before getting info of such kinf of vacations lets prepare everything required for this analysis.

In [1]:
#Lets import modules (Dependencies)
import pandas as pd
import requests
import json
import gmaps
import numpy as np
import matplotlib as plt

#Lets import the api key
from config import g_key

#Lets configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
#Lets call the df with weather info
cities_path = "output_data/cities_weather.csv"

In [3]:
cities_df = pd.read_csv(cities_path)
cities_df.head()

,City,Country,Longitude,Latitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind Speed
0,butaritari,KI,172.79,3.07,80.10,80.10,81,99,2.28
1,hermanus,ZA,19.23,-34.42,60.01,60.01,89,5,2.51
2,batemans bay,AU,150.18,-35.72,53.17,55.00,96,100,4.00
3,bubaque,GW,-15.83,11.28,80.64,80.64,81,34,11.45
4,faanui,PF,-151.75,-16.48,77.72,77.72,77,6,16.82


In [4]:
#Lets see the size
cities_df.shape

(593, 9)

In [5]:
#Check coordinates
print(f"The maximum latitude is {cities_df['Latitude'].max()} and the minimum is {cities_df['Latitude'].min()}")
print(f"The maximum longitude is {cities_df['Longitude'].max()} and the minimum is {cities_df['Longitude'].min()}")

The maximum latitude is 78.22 and the minimum is -54.8
The maximum longitude is 179.32 and the minimum is -175.2


## Heat map of world's humidity(08-09-2020)
- - - -

To generate a heatmap, whose colors are going to be defined bu the humidity of each of the cities in the dataframe, the gmaps API. Gmaps is a plugin for Jupyter for embedding Google Maps in a notebook, and is designed as a data visualization tool.

In order to generate a mao is necessary to use the function gmaps.figure, and the it is necessary to add a layer that contains a function named gmaps.heatmap_layer. Is in this last fucntion where the geolocation coordinates and the variable that is going to define intensity in each geolocation of the heat map. 

Lets create the humidity heat map.

In [6]:
#Lets check what kind of elemets are in cities_df
cities_df.dtypes

City                object
Country             object
Longitude          float64
Latitude           float64
Temperature        float64
Max Temperature    float64
Humidity             int64
Cloudiness           int64
Wind Speed         float64
dtype: object

In [7]:
#Since all the variables are in the correct format no transformation has tobe done
#Lets extract the coordinates
# Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]

# Store humidity in humidity
humidity = cities_df["Humidity"]

# Keep maximum humidity
max_hum = cities_df["Humidity"].max()

In [8]:
#Lets create the map
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity= max_hum, point_radius=3.0)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

## Selecting the best cities for an idillyc vacation
- - - -

There is a concept known as Thermal comfort, and is defined as the condition of mind that expresses satisfaction with the thermal environment and is assessed by subjective evaluation (wikipedia, ANSI/ASHRAE Standard 55). Accoridng to to Covarrubias (2012), the temperature confort range is between 21.1°C and 27.5°, but other authors propose a range between 18°c abd 20°. Also, Covarrubias (2012) metions an international standard and an ISO norm that states that in order to keep air quality inside a building humidity must be between  dty that make  that is between 30% and 70%. With respect to wind speed, the web page (https://www.seiscubos.com/conocimiento/confort-termico-y-cuerpo-humano) at templated climates the best wind speed is at is less than 0.25m/s. Therefore, in this section cities with weather parameters in hte explianed above are going to be selected. In the next table, the ranges as specified in the Imperial Units. Also only cities without a low amount of clouds are going to be selected

| Variable | minimun | maximum |
| - - - - -|- - - - -| - - - - |
|Temperature | 69.98 F| 81.5  F |
|Humidity  | 30 %    | 70%     |
|Wind Speed| 0.56 mph| 10 mph  |
|Cloudiness| 0 %     | 10 %    |




In [9]:
#Lets filter cities_df by maximum values
vacation_df = cities_df[((cities_df["Max Temperature"] <= 81.50) & (cities_df["Humidity"] <= 70) & (cities_df["Wind Speed"] <= 10) & (cities_df["Cloudiness"] <= 10))]
vacation_df.shape
                  

(41, 9)

In [10]:
#Lets filter vacatios df by minimum values
vacation_df = vacation_df[((vacation_df["Max Temperature"] >= 69.98) & (vacation_df["Humidity"] >= 30) & (vacation_df["Wind Speed"] >= 0.56))]
vacation_df.shape

(7, 9)

In [ ]:
vacation_df

## Searching for a hotel in the best cities for vacation
- - - - 

Now, it is going to be searched if in the 17 cities selected is an hotel. To do so the Google Places API is going to be used. The selested kind of query is a Nearby Search requests.  A Nearby Search allows to search for places within a specified area. It is possible to refine the search request by supplying keywords or specifying the type of place serached for.

A Nearby Search request is an HTTP URL of the following form:

https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters

In our case the ouput is a json file. Also, the required parameters are the API key, a keyword that os a term to be matched against all content that Google has indexed for a place. A name which is a a term to be matched against all content that Google has indexed for a place and is equivalent  to keyword. opennow, returns only those places that are open for business at the time the query is sent, rankby specifies the order in which results are listed, etc.

In this case it is going to be used the following parameters: "radius", "types", and the"key".

In the next cells the procedure is shown.


In [11]:
##Lets start the query for the vactions_df
#To do so, first we need to add new columns to vactions_df, where info is going to be stores
# Add columns for lat, lng, airport name, airport address, airport rating
# Note that we used "" to specify initial entry.
vacation_df["Hotel Name"] = ""
vacation_df["Hotel Address"] = ""
vacation_df.head()

,City,Country,Longitude,Latitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name,Hotel Address
23,westport,US,-73.36,41.14,78.78,81.0,57,1,5.82,,
205,port hedland,AU,118.57,-20.32,71.60,71.6,56,0,9.17,,
304,alghero,IT,8.32,40.56,78.80,78.8,69,1,4.70,,
455,yulara,AU,130.99,-25.24,71.60,71.6,35,0,8.05,,
522,tucuman,AR,-65.22,-26.82,78.80,78.8,39,0,9.17,,


In [12]:
# Lets define the query

#parameters dictionary to update each iteration the keyword lodgind is added to ensure it is a hotel and not another kind of 
#bussines that might have in its name the world hotel
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to search for hotels
for index, row in vacation_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request
    name_address = requests.get(base_url, params=params)
    

    # convert to json
    name_address = name_address.json()
    

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        vacation_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        vacation_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


In [13]:
#Lets se the result
vacation_df

,City,Country,Longitude,Latitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name,Hotel Address
23,westport,US,-73.36,41.14,78.78,81.0,57,1,5.82,The Guesthouse at the Norwalk Inn,"93 East Ave, Norwalk"
205,port hedland,AU,118.57,-20.32,71.60,71.6,56,0,9.17,The Esplanade Hotel,"2 Anderson St, Port Hedland"
304,alghero,IT,8.32,40.56,78.80,78.8,69,1,4.70,Villa Las Tronas,"Lungomare Valencia, 1, Alghero"
455,yulara,AU,130.99,-25.24,71.60,71.6,35,0,8.05,Sails in the Desert,"163 Yulara Dr, Yulara"
522,tucuman,AR,-65.22,-26.82,78.80,78.8,39,0,9.17,"CiTá, bed and breakfasts","Gral. Lamadrid 1457, San Miguel de Tucumán"
560,bad kreuznach,DE,7.87,49.83,69.76,72.0,39,1,2.73,PK Parkhotel Kurhaus,"Kurhausstraße 28, Bad Kreuznach"
567,kotel,BG,26.45,42.88,76.53,77.0,61,5,9.17,Yovina House,"Zheravna, 59, Zheravna"


## Pointing the location of an hotel in the selected cities
- - - - 

In this last part, a layer to the humidity map is added. The layer is goingo to contain markers with the location of some of the cities where aj hotel is found from the last request. To do a list of a messages that are going to be shown in each marker is created. The result is shown in the next figure.

In [14]:
#Lets convert the missing values into NAN objects 
vacation_df.replace("", np.nan, inplace=True) 
vacation_df

,City,Country,Longitude,Latitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name,Hotel Address
23,westport,US,-73.36,41.14,78.78,81.0,57,1,5.82,The Guesthouse at the Norwalk Inn,"93 East Ave, Norwalk"
205,port hedland,AU,118.57,-20.32,71.60,71.6,56,0,9.17,The Esplanade Hotel,"2 Anderson St, Port Hedland"
304,alghero,IT,8.32,40.56,78.80,78.8,69,1,4.70,Villa Las Tronas,"Lungomare Valencia, 1, Alghero"
455,yulara,AU,130.99,-25.24,71.60,71.6,35,0,8.05,Sails in the Desert,"163 Yulara Dr, Yulara"
522,tucuman,AR,-65.22,-26.82,78.80,78.8,39,0,9.17,"CiTá, bed and breakfasts","Gral. Lamadrid 1457, San Miguel de Tucumán"
560,bad kreuznach,DE,7.87,49.83,69.76,72.0,39,1,2.73,PK Parkhotel Kurhaus,"Kurhausstraße 28, Bad Kreuznach"
567,kotel,BG,26.45,42.88,76.53,77.0,61,5,9.17,Yovina House,"Zheravna, 59, Zheravna"


In [15]:
#Lets drop nas from de vacation_df
vacation_df.dropna(subset=['Hotel Name'], inplace=True)
vacation_df 

,City,Country,Longitude,Latitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name,Hotel Address
23,westport,US,-73.36,41.14,78.78,81.0,57,1,5.82,The Guesthouse at the Norwalk Inn,"93 East Ave, Norwalk"
205,port hedland,AU,118.57,-20.32,71.60,71.6,56,0,9.17,The Esplanade Hotel,"2 Anderson St, Port Hedland"
304,alghero,IT,8.32,40.56,78.80,78.8,69,1,4.70,Villa Las Tronas,"Lungomare Valencia, 1, Alghero"
455,yulara,AU,130.99,-25.24,71.60,71.6,35,0,8.05,Sails in the Desert,"163 Yulara Dr, Yulara"
522,tucuman,AR,-65.22,-26.82,78.80,78.8,39,0,9.17,"CiTá, bed and breakfasts","Gral. Lamadrid 1457, San Miguel de Tucumán"
560,bad kreuznach,DE,7.87,49.83,69.76,72.0,39,1,2.73,PK Parkhotel Kurhaus,"Kurhausstraße 28, Bad Kreuznach"
567,kotel,BG,26.45,42.88,76.53,77.0,61,5,9.17,Yovina House,"Zheravna, 59, Zheravna"


In [17]:
#Create a list with message to display
messages =[]

#Iterate by rows
for index, row in vacation_df.iterrows():
    # get info
    hotel = row["Hotel Name"]
    city = row["City"].capitalize()
    country = row["Country"]
    address = row["Hotel Address"]
    #append to the messages list
    messages.append(f"Name: {hotel}, City: {city}, Address: {address}, Country : {country}")

messages

['Name: The Guesthouse at the Norwalk Inn, City: Westport, Address: 93 East Ave, Norwalk, Country : US',
 'Name: The Esplanade Hotel, City: Port hedland, Address: 2 Anderson St, Port Hedland, Country : AU',
 'Name: Villa Las Tronas, City: Alghero, Address: Lungomare Valencia, 1, Alghero, Country : IT',
 'Name: Sails in the Desert, City: Yulara, Address: 163 Yulara Dr, Yulara, Country : AU',
 'Name: CiTá, bed and breakfasts, City: Tucuman, Address: Gral. Lamadrid 1457, San Miguel de Tucumán, Country : AR',
 'Name: PK Parkhotel Kurhaus, City: Bad kreuznach, Address: Kurhausstraße 28, Bad Kreuznach, Country : DE',
 'Name: Yovina House, City: Kotel, Address: Zheravna, 59, Zheravna, Country : BG']

In [18]:
#Lets create the layer

#Create position of markers
markers_location = vacation_df[["Latitude", "Longitude"]]

#Create the layer
markers = gmaps.marker_layer(markers_location, info_box_content= [f"{message}" for message in messages])

                                                          
#Lets add the layer to the last map
fig.add_layer(markers)


In [19]:
fig

Figure(layout=FigureLayout(height='420px'))